In [1]:
from model.finetuned_bert_model import FineTunedBertModel

In [2]:
train_file_name = 'train.csv'
test_file_name = 'test.csv'
sample_submission_file_name = 'sample_submission.csv'

In [3]:
# from data_preparation.utils import DataHandler
#
# data_handler = DataHandler(train_file_name=train_file_name, test_file_name=test_file_name, sample_submission_file_name=sample_submission_file_name)

In [4]:
max_length = 60
model = FineTunedBertModel(train_file_name=train_file_name, test_file_name=test_file_name, sample_submission_file_name=sample_submission_file_name)

Bert tokenizer: bert-base-uncased
Getting the file: /home/sersery/Desktop/TUM Informatik/WS2122/Applied Deep Learning for NLP/KAGGLE-nlp-with-disaster-tweets/data/train.csv
Getting the file: /home/sersery/Desktop/TUM Informatik/WS2122/Applied Deep Learning for NLP/KAGGLE-nlp-with-disaster-tweets/data/test.csv
Getting the file: /home/sersery/Desktop/TUM Informatik/WS2122/Applied Deep Learning for NLP/KAGGLE-nlp-with-disaster-tweets/data/sample_submission.csv
Training dataset inputs: (array([[  101,  2256, 15616, ...,     0,     0,     0],
       [  101,  3224,  2543, ...,     0,     0,     0],
       [  101,  2035,  3901, ...,     0,     0,     0],
       ...,
       [  101, 23290,  1012, ...,     0,     0,     0],
       [  101,  2610, 11538, ...,     0,     0,     0],
       [  101,  1996,  6745, ...,     0,     0,     0]]), array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 

INFO:absl:Using /tmp/tfhub_modules to cache modules.
INFO:absl:Downloading TF-Hub Module 'https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1'.
INFO:absl:Downloading https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1: 50.00MB
INFO:absl:Downloading https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1: 90.00MB
INFO:absl:Downloading https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1: 130.00MB
INFO:absl:Downloading https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1: 180.00MB
INFO:absl:Downloading https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1: 220.00MB
INFO:absl:Downloading https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1: 250.00MB
INFO:absl:Downloading https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1: 290.00MB
INFO:absl:Downloading https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1: 320.00MB
INFO:absl:Downloading https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024

In [5]:
model.build(max_length=max_length)

****************** HYPERPARAMETERS ******************
Optimizer: <keras.optimizer_v2.adam.Adam object at 0x7fabb0465ca0>
Learning rate: 1e-05
Batch size: 16
******************************************************
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 60)]         0           []                               
                                                                                                  
 input_mask (InputLayer)        [(None, 60)]         0           []                               
                                                                                                  
 segment_ids (InputLayer)       [(None, 60)]         0           []                               
                                                                                

In [6]:
epochs = 3
validation_split = 0.2
model.fit(epochs=epochs, validation_split=validation_split)

Epoch 1/3
381/381 [==============================] - 5229s 14s/step - loss: 0.8210 - accuracy: 0.5479 - val_loss: 0.6981 - val_accuracy: 0.5345
Epoch 2/3
381/381 [==============================] - 4912s 13s/step - loss: 0.6980 - accuracy: 0.5504 - val_loss: 0.8312 - val_accuracy: 0.5345
Epoch 3/3
381/381 [==============================] - 4957s 13s/step - loss: 0.6911 - accuracy: 0.5626 - val_loss: 0.6909 - val_accuracy: 0.5345
